In [11]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import pandas as pd

X = np.loadtxt('X.txt')
y = np.loadtxt('y.txt')

In [12]:
##
# X: 2d array with the input features
# y: 1d array with the class labels (0 or 1)
#
def plot_data_internal(X, y,title=''):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    plt.figure(figsize=(10,10))
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    ax = plt.gca()
    ax.plot(X[y == 0, 0], X[y == 0, 1], 'ro', markersize=2, label = 'Class 1')
    ax.plot(X[y == 1, 0], X[y == 1, 1], 'bo', markersize=2, label = 'Class 2')
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.title('Plot data '+title)
    plt.legend(loc = 'upper left', scatterpoints = 1, numpoints = 1)
    return xx, yy

##
# X: 2d array with the input features
# y: 1d array with the class labels (0 or 1)
#
def plot_data(X, y):
    xx, yy = plot_data_internal(X, y)
    plt.show()

##
# x: input to the logistic function
#
def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

##
# X: 2d array with the input features
# y: 1d array with the class labels (0 or 1)
# w: current parameter values
#
def compute_average_ll(X, y, w):
    output_prob = logistic(np.dot(X, w))
    return np.mean(y * np.log(output_prob)
                   + (1 - y) * np.log(1.0 - output_prob))

def compute_average_ll_fixed(X, y, w):
    return np.mean(y * np.log(logistic(np.dot(X, w)))
                   + (1 - y) * np.log(logistic(-np.dot(X, w))))

##
# ll: 1d array with the average likelihood per data point, for each training
# step. The dimension of this array should be equal to the number of training
# steps.
#
def plot_ll(ll):
    plt.figure()
    ax = plt.gca()
    plt.xlim(0, len(ll) + 2)
    plt.ylim(min(ll) - 0.1, max(ll) + 0.1)
    ax.plot(np.arange(1, len(ll) + 1), ll, 'r-')
    plt.xlabel('Steps')
    plt.ylabel('Average log-likelihood')
    plt.title('Plot Average Log-likelihood Curve')
    plt.show()

##
# X: 2d array with input features at which to compute predictions.
#(uses parameter vector w which is defined outside the function's scope)
#
def predict_for_plot(x):
    x_tilde = np.concatenate((x, np.ones((x.shape[ 0 ], 1 ))), 1)
    return logistic(np.dot(x_tilde, w))

##
# X: 2d array with the input features
# y: 1d array with the class labels (0 or 1)
# predict: function that recives as input a feature matrix and returns a 1d
#          vector with the probability of class 1.
def plot_predictive_distribution(X, y, predict,title):
    xx, yy = plot_data_internal(X, y,title)
    ax = plt.gca()
    X_predict = np.concatenate((xx.ravel().reshape((-1, 1)),
                                yy.ravel().reshape((-1, 1))), 1)
    Z = predict(X_predict)
    Z = Z.reshape(xx.shape)
    cs2 = ax.contour(xx, yy, Z, cmap = 'RdBu', linewidths = 2, levels=[0.1,0.3,0.5,0.7,0.9])
    plt.clabel(cs2, fmt = '%2.1f', colors = 'k', fontsize = 14)
    plt.imshow(Z,interpolation="bilinear", origin="lower", cmap="RdBu", extent=(np.amin(xx), np.amax(xx), np.amin(yy), np.amax(yy)), zorder=0)
    plt.show()

##
# l: hyper-parameter for the width of the Gaussian basis functions
# Z: location of the Gaussian basis functions
# X: points at which to evaluate the basis functions
def expand_inputs(l, X, Z):
    X2 = np.sum(X**2, 1)
    Z2 = np.sum(Z**2, 1)
    ones_Z = np.ones(Z.shape[ 0 ])
    ones_X = np.ones(X.shape[ 0 ])
    r2 = np.outer(X2, ones_Z) - 2 * np.dot(X, Z.T) + np.outer(ones_X, Z2)
    return np.exp(-0.5 / l**2 * r2)

##
# x: 2d array with input features at which to compute the predictions
# using the feature expansion
#
# (uses parameter vector w and the 2d array X with the centers of the basis
# functions for the feature expansion, which are defined outside the function's
# scope)
#
def predict_for_plot_expanded_features(x):
    x_expanded = expand_inputs(l, x, C)
    x_tilde = np.concatenate((x_expanded, np.ones((x_expanded.shape[ 0 ], 1 ))), 1)
    return logistic(np.dot(x_tilde, w_MAP))

# New functions

In [13]:
def neg_log_posterior_not_normalized(w):
    output_prob = logistic(np.dot(X_train, w))
    log_likelihood = np.dot(y_train, np.log(output_prob)) + np.dot((1 - y_train),np.log(1.0 - output_prob))
    log_prior_without_const = -0.5 * np.dot(w,w)/prior_var
    return -(log_likelihood + log_prior_without_const)

def grad_neg_log_posterior(w):
    output_prob = logistic(np.dot(X_train, w))
    grad_log_prior = -w/(prior_var)
    grad_log_likelihood = np.dot(y_train-output_prob.T, X_train)
    return -(grad_log_prior+grad_log_likelihood)

def predict_for_full_bayesian(x):
    results = np.zeros(x.shape[0])
    x_expanded = expand_inputs(l, x, C)
    x_tilde = np.concatenate((x_expanded, np.ones((x_expanded.shape[ 0 ], 1 ))), 1)
#     print(x_tilde.shape)
#     print(x_tilde[0].shape)
    for i in range(x.shape[0]):
        mu_a = np.dot(w_MAP, np.transpose(x_tilde[i]))
    

        sigma_a = np.dot(x_tilde[i],np.dot(S_posterior,np.transpose(x_tilde[i])))

        kappa = (1 + np.pi*sigma_a/8)**-0.5
        results[i] = logistic(np.dot(kappa,mu_a))
    return results

def compute_average_lme(X,y,w_MAP,S_posterior,prior_var):
    log_likelihood = ( y * np.log(logistic(np.dot(X, w_MAP)))
                   + (1 - y) * np.log(logistic(-np.dot(X, w_MAP))))
    print('log likelihood is', np.sum(log_likelihood))
    
    occam_1 = -0.5*np.dot(w_MAP,w_MAP)/(prior_var)
    print('occam 1 is', occam_1)
    
    occam_2 = -0.5*w_MAP.size*np.log(prior_var)
    print('occam 2 is', occam_2)
    
    occam_3 = 0.5*np.linalg.slogdet(S_posterior)[1]
    print('occam 3 is', occam_3)
    
    print('log model evidence is', np.sum(log_likelihood)+occam_1+occam_2+occam_3)
    return np.sum(log_likelihood)+occam_1+occam_2+occam_3

def compute_average_bayes_ll(X, y, w,S):
    output_prob = np.zeros(y.size)
    for i in range(len(output_prob)):
        mu_a = np.dot(w, X[i,:])
    
        sigma_a = np.dot(X[i,:],np.dot(S,X[i,:]))

        kappa = (1 + np.pi*sigma_a/8)**-0.5
        
        output_prob[i] = logistic(np.dot(kappa,mu_a))
    results = y*np.log(output_prob) + (1-y)*np.log(output_prob)
    return np.mean(results)

In [14]:
def plot_conditional_posterior():
    w_for_plot = np.zeros([100,w_MAP.size])
    count = 0
    for i in np.linspace(-1,1,100):
        w_for_plot[count] = np.concatenate([np.array([i]),w_MAP[1:]])
        count += 1
    prob_for_plot = [np.exp(log_posterior(w_for_plot[i])) for i in range(len(w_for_plot))]
    prob_for_plot = np.array(prob_for_plot)

    plt.plot(np.linspace(-1,1,100),prob_for_plot)
    plt.show()

# Results on l = 0.1 and variance = 1

In [15]:
ls = np.array([0.1])
variances = np.array([1])

grid5 = np.zeros([ls.size,variances.size])

for l_index in range(ls.size):
    for var_index in range(variances.size):
        print("RUN", l_index*variances.size + var_index)
        #Prior hyperparameters
        prior_mean = 0
        prior_var = variances[var_index]

        #RBF width
        l = ls[l_index]        
        
        print('## l   =',l)
        print('## var =',prior_var)
        
        #Train test split
        train_portion = 0.8
        train_size = int(np.size(X,0)*train_portion)

        #RBF centres
        C = X[:train_size,:]

        # #RBF train set
        X_train = expand_inputs(l,X[:train_size,:],C)
        X_train = np.column_stack([X_train,np.ones(X_train.shape[0])])
        y_train = y[:train_size]

        # #RBF test set
        X_test = expand_inputs(l,X[train_size:,:],C)
        X_test = np.column_stack([X_test,np.ones(X_test.shape[0])])
        y_test = y[train_size:]

        #Initialise Gaussian prior
        np.random.seed(42)
        w_prior = np.random.normal(prior_mean,prior_var**0.5,X_train.shape[1])
        w = w_prior
        S_prior = prior_var * np.eye(w.size)

        #MAP with bfgs
        w_MAP = optimize.fmin_l_bfgs_b(neg_log_posterior_not_normalized, w_prior, fprime=grad_neg_log_posterior)[0]
        #Calculate S_posterior
        Hessian =  np.linalg.inv(S_prior)
        Hessian += sum(logistic(np.dot(X_train[i],w_MAP)) * (1-logistic(np.dot(X_train[i],w_MAP))) * np.outer(X_train[i],X_train[i]) for i in range(X_train.shape[0]))
        S_posterior = np.linalg.inv(Hessian)

    #         if np.all(np.linalg.eigvals(S_posterior) > 0): print('S_posterior is positive definite')

        plot_predictive_distribution(X,y,predict_for_full_bayesian,'Full Bayesian')
        plot_predictive_distribution(X,y,predict_for_plot_expanded_features,'MAP')

        print('___FULL BAYESIAN___')
        print('TRAIN LL:',compute_average_bayes_ll(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_bayes_ll(X_test,y_test,w_MAP,S_posterior))

        print('\n___MAP___')
        print('TRAIN LL:',compute_average_ll_fixed(X_train,y_train,w_MAP))
    #         print('TRAIN LOG MODEL EVIDENCE:',compute_average_lme(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_ll_fixed(X_test,y_test,w_MAP))
    #         print('TEST LOG MODEL EVIDENCE:',compute_average_lme(X_test,y_test,w_MAP,S_posterior))

        print('\n___TEST MODEL EVIDENCE___:')
        compute_average_lme(X_test,y_test,w_MAP,S_posterior,prior_var)

        TP, TN, FP, FN = 0,0,0,0
        for i in range(len(X_test)):
            mu_a = np.dot(w_MAP, X_test[i,:])
            sigma_a = np.dot(X_test[i,:],np.dot(S_posterior,X_test[i,:]))
            kappa = (1 + np.pi*sigma_a/8)**-0.5

            prediction = 1 if logistic(kappa*mu_a)>0.5 else 0
            if prediction == y_test[i]:
                if y_test[i]==0: TN += 1
                elif y_test[i]==1: TP += 1
            else:
                if y_test[i]==0: FP += 1
                elif y_test[i]==1: FN += 1
        print('===FB CONFUSION MATRIX===')
        print(TN/(TN+FP), FP/(TN+FP))
        print(FN/(FN+TP), TP/(FN+TP))
        
        
        TP, TN, FP, FN = 0,0,0,0
        for i in range(len(X_test)):
            prediction = 1 if logistic(np.dot(X_test[i],w_MAP))>0.5 else 0
            if prediction == y_test[i]:
                if y_test[i]==0: TN += 1
                elif y_test[i]==1: TP += 1
            else:
                if y_test[i]==0: FP += 1
                elif y_test[i]==1: FN += 1
        print('===CONFUSION MATRIX===')
        print(TN/(TN+FP), FP/(TN+FP))
        print(FN/(FN+TP), TP/(FN+TP))

        print("______________________________\n\n")


RUN 0
## l   = 0.1
## var = 1
___FULL BAYESIAN___
TRAIN LL: -1.0041454429145582
TEST LL: -0.9292464317323779

___MAP___
TRAIN LL: -0.21659107076658657
TEST LL: -0.3242690355101158

___TEST MODEL EVIDENCE___:
log likelihood is -64.85380710202317
occam 1 is -65.88356907242733
occam 2 is -0.0
occam 3 is -78.01973970579348
log model evidence is -208.757115880244
===FB CONFUSION MATRIX===
0.9108910891089109 0.0891089108910891
0.1414141414141414 0.8585858585858586
===CONFUSION MATRIX===
0.9108910891089109 0.0891089108910891
0.1414141414141414 0.8585858585858586
______________________________




# First grid search

In [ ]:
# ls = np.linspace(0.01,10,5)
# variances = np.linspace(.1,10000,5)
ls = np.array([0.01,0.1,1,10])
variances = np.array([0.1,1,10,100,1000,10000])

grid1 = np.zeros([ls.size,variances.size])

for l_index in range(ls.size):
    for var_index in range(variances.size):
        print("RUN", l_index*variances.size + var_index)
        #Prior hyperparameters
        prior_mean = 0
        prior_var = variances[var_index]

        #RBF width
        l = ls[l_index]        
        
        print('## l   =',l)
        print('## var =',prior_var)
        
        #Train test split
        train_portion = 0.8
        train_size = int(np.size(X,0)*train_portion)

        #RBF centres
        C = X[:train_size,:]

        # #RBF train set
        X_train = expand_inputs(l,X[:train_size,:],C)
        X_train = np.column_stack([X_train,np.ones(X_train.shape[0])])
        y_train = y[:train_size]

        # #RBF test set
        X_test = expand_inputs(l,X[train_size:,:],C)
        X_test = np.column_stack([X_test,np.ones(X_test.shape[0])])
        y_test = y[train_size:]
        
        #Initialise Gaussian prior
        np.random.seed(42)
        w_prior = np.random.normal(prior_mean,prior_var**0.5,X_train.shape[1])
        w = w_prior
        S_prior = prior_var * np.eye(w.size)

        #MAP with bfgs
        w_MAP = optimize.fmin_l_bfgs_b(neg_log_posterior_not_normalized, w_prior, fprime=grad_neg_log_posterior)[0]
        #Calculate S_posterior
        Hessian =  np.linalg.inv(S_prior)
        Hessian += sum(logistic(np.dot(X_train[i],w_MAP)) * (1-logistic(np.dot(X_train[i],w_MAP))) * np.outer(X_train[i],X_train[i]) for i in range(X_train.shape[0]))
        S_posterior = np.linalg.inv(Hessian)

#         if np.all(np.linalg.eigvals(S_posterior) > 0): print('S_posterior is positive definite')
        
        plot_predictive_distribution(X,y,predict_for_full_bayesian,'Full Bayesian')
        plot_predictive_distribution(X,y,predict_for_plot_expanded_features,'MAP')
        
        print('___FULL BAYESIAN___')
        print('TRAIN LL:',compute_average_bayes_ll(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_bayes_ll(X_test,y_test,w_MAP,S_posterior))
        
        print('\n___MAP___')
        print('TRAIN LL:',compute_average_ll_fixed(X_train,y_train,w_MAP))
#         print('TRAIN LOG MODEL EVIDENCE:',compute_average_lme(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_ll_fixed(X_test,y_test,w_MAP))
#         print('TEST LOG MODEL EVIDENCE:',compute_average_lme(X_test,y_test,w_MAP,S_posterior))

        print('\n___TEST MODEL EVIDENCE___:')
        grid1[l_index,var_index] = compute_average_lme(X_test,y_test,w_MAP,S_posterior,prior_var)
        
        TP, TN, FP, FN = 0,0,0,0
        for i in range(len(X_test)):
            mu_a = np.dot(w_MAP, X_test[i,:])
            sigma_a = np.dot(X_test[i,:],np.dot(S_posterior,X_test[i,:]))
            kappa = (1 + np.pi*sigma_a/8)**-0.5
            
            prediction = 1 if logistic(kappa*mu_a)>0.5 else 0
            if prediction == y_test[i]:
                if y_test[i]==0: TN += 1
                elif y_test[i]==1: TP += 1
            else:
                if y_test[i]==0: FP += 1
                elif y_test[i]==1: FN += 1
        print('===CONFUSION MATRIX===')
        print(TN/(TN+FP), FP/(TN+FP))
        print(FN/(FN+TP), TP/(FN+TP))
        
        print("______________________________\n\n")

## First grid search results

In [ ]:
grid_df = pd.DataFrame(data = np.round(grid1,2), index=ls, columns=variances)
grid_df

In [ ]:
grid_df.to_latex('grid1.tex')

In [ ]:
grid1.max()

# Second grid search

In [ ]:
ls = np.linspace(0.1,10,10)
variances = np.linspace(1,100,10)
# ls = np.array([0.1,1,10])
# variances = np.array([0.1,1,10])

grid2 = np.zeros([ls.size,variances.size])

for l_index in range(ls.size):
    for var_index in range(variances.size):
        print("RUN", l_index*variances.size + var_index)
        #Prior hyperparameters
        prior_mean = 0
        prior_var = variances[var_index]

        #RBF width
        l = ls[l_index]        
        
        print('## l   =',l)
        print('## var =',prior_var)
        
        #Train test split
        train_portion = 0.8
        train_size = int(np.size(X,0)*train_portion)

        #RBF centres
        C = X[:train_size,:]

        # #RBF train set
        X_train = expand_inputs(l,X[:train_size,:],C)
        X_train = np.column_stack([X_train,np.ones(X_train.shape[0])])
        y_train = y[:train_size]

        # #RBF test set
        X_test = expand_inputs(l,X[train_size:,:],C)
        X_test = np.column_stack([X_test,np.ones(X_test.shape[0])])
        y_test = y[train_size:]
        
        #Initialise Gaussian prior
        np.random.seed(42)
        w_prior = np.random.normal(prior_mean,prior_var**0.5,X_train.shape[1])
        w = w_prior
        S_prior = prior_var * np.eye(w.size)

        #MAP with bfgs
        w_MAP = optimize.fmin_l_bfgs_b(neg_log_posterior_not_normalized, w_prior, fprime=grad_neg_log_posterior)[0]
        #Calculate S_posterior
        Hessian =  np.linalg.inv(S_prior)
        Hessian += sum(logistic(np.dot(X_train[i],w_MAP)) * (1-logistic(np.dot(X_train[i],w_MAP))) * np.outer(X_train[i],X_train[i]) for i in range(X_train.shape[0]))
        S_posterior = np.linalg.inv(Hessian)

#         if np.all(np.linalg.eigvals(S_posterior) > 0): print('S_posterior is positive definite')
        
        plot_predictive_distribution(X,y,predict_for_full_bayesian,'Full Bayesian')
        plot_predictive_distribution(X,y,predict_for_plot_expanded_features,'MAP')
        
        print('___FULL BAYESIAN___')
        print('TRAIN LL:',compute_average_bayes_ll(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_bayes_ll(X_test,y_test,w_MAP,S_posterior))
        
        print('\n___MAP___')
        print('TRAIN LL:',compute_average_ll_fixed(X_train,y_train,w_MAP))
#         print('TRAIN LOG MODEL EVIDENCE:',compute_average_lme(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_ll_fixed(X_test,y_test,w_MAP))
#         print('TEST LOG MODEL EVIDENCE:',compute_average_lme(X_test,y_test,w_MAP,S_posterior))

        print('\n___TEST MODEL EVIDENCE___:')
        grid2[l_index,var_index] = compute_average_lme(X_test,y_test,w_MAP,S_posterior,prior_var)
        
        TP, TN, FP, FN = 0,0,0,0
        for i in range(len(X_test)):
            mu_a = np.dot(w_MAP, X_test[i,:])
            sigma_a = np.dot(X_test[i,:],np.dot(S_posterior,X_test[i,:]))
            kappa = (1 + np.pi*sigma_a/8)**-0.5
            
            prediction = 1 if logistic(kappa*mu_a)>0.5 else 0
            if prediction == y_test[i]:
                if y_test[i]==0: TN += 1
                elif y_test[i]==1: TP += 1
            else:
                if y_test[i]==0: FP += 1
                elif y_test[i]==1: FN += 1
        print('===CONFUSION MATRIX===')
        print(TN/(TN+FP), FP/(TN+FP))
        print(FN/(FN+TP), TP/(FN+TP))
        
        print("______________________________\n\n")

## Second grid search results

In [ ]:
grid_df = pd.DataFrame(data = np.round(grid2,2), index=ls, columns=variances)
grid_df

In [ ]:
grid_df.to_latex('grid2.tex')

In [ ]:
grid2.max()

# 3rd grid search

In [ ]:
ls = np.linspace(0.1,2.3,10)
variances = np.linspace(1,23,10)
# ls = np.array([0.1,1,10])
# variances = np.array([0.1,1,10])

grid3 = np.zeros([ls.size,variances.size])

for l_index in range(ls.size):
    for var_index in range(variances.size):
        print("RUN", l_index*variances.size + var_index)
        #Prior hyperparameters
        prior_mean = 0
        prior_var = variances[var_index]

        #RBF width
        l = ls[l_index]        
        
        print('## l   =',l)
        print('## var =',prior_var)
        
        #Train test split
        train_portion = 0.8
        train_size = int(np.size(X,0)*train_portion)

        #RBF centres
        C = X[:train_size,:]

        # #RBF train set
        X_train = expand_inputs(l,X[:train_size,:],C)
        X_train = np.column_stack([X_train,np.ones(X_train.shape[0])])
        y_train = y[:train_size]

        # #RBF test set
        X_test = expand_inputs(l,X[train_size:,:],C)
        X_test = np.column_stack([X_test,np.ones(X_test.shape[0])])
        y_test = y[train_size:]
        
        #Initialise Gaussian prior
        np.random.seed(42)
        w_prior = np.random.normal(prior_mean,prior_var**0.5,X_train.shape[1])
        w = w_prior
        S_prior = prior_var * np.eye(w.size)

        #MAP with bfgs
        w_MAP = optimize.fmin_l_bfgs_b(neg_log_posterior_not_normalized, w_prior, fprime=grad_neg_log_posterior)[0]
        #Calculate S_posterior
        Hessian =  np.linalg.inv(S_prior)
        Hessian += sum(logistic(np.dot(X_train[i],w_MAP)) * (1-logistic(np.dot(X_train[i],w_MAP))) * np.outer(X_train[i],X_train[i]) for i in range(X_train.shape[0]))
        S_posterior = np.linalg.inv(Hessian)

#         if np.all(np.linalg.eigvals(S_posterior) > 0): print('S_posterior is positive definite')
        
        plot_predictive_distribution(X,y,predict_for_full_bayesian,'Full Bayesian')
        plot_predictive_distribution(X,y,predict_for_plot_expanded_features,'MAP')
        
        print('___FULL BAYESIAN___')
        print('TRAIN LL:',compute_average_bayes_ll(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_bayes_ll(X_test,y_test,w_MAP,S_posterior))
        
        print('\n___MAP___')
        print('TRAIN LL:',compute_average_ll_fixed(X_train,y_train,w_MAP))
#         print('TRAIN LOG MODEL EVIDENCE:',compute_average_lme(X_train,y_train,w_MAP,S_posterior))
        print('TEST LL:',compute_average_ll_fixed(X_test,y_test,w_MAP))
#         print('TEST LOG MODEL EVIDENCE:',compute_average_lme(X_test,y_test,w_MAP,S_posterior))

        print('\n___TEST MODEL EVIDENCE___:')
        grid3[l_index,var_index] = compute_average_lme(X_test,y_test,w_MAP,S_posterior,prior_var)
        
        TP, TN, FP, FN = 0,0,0,0
        for i in range(len(X_test)):
            mu_a = np.dot(w_MAP, X_test[i,:])
            sigma_a = np.dot(X_test[i,:],np.dot(S_posterior,X_test[i,:]))
            kappa = (1 + np.pi*sigma_a/8)**-0.5
            
            prediction = 1 if logistic(kappa*mu_a)>0.5 else 0
            if prediction == y_test[i]:
                if y_test[i]==0: TN += 1
                elif y_test[i]==1: TP += 1
            else:
                if y_test[i]==0: FP += 1
                elif y_test[i]==1: FN += 1
        print('===CONFUSION MATRIX===')
        print(TN/(TN+FP), FP/(TN+FP))
        print(FN/(FN+TP), TP/(FN+TP))
        
        print("______________________________\n\n")

# 3rd grid search results

In [ ]:
grid_df = pd.DataFrame(data = np.round(grid3,2), index=np.round(ls,2), columns=np.round(variances,2))
grid_df

In [ ]:
grid3.max()

In [ ]:
grid_df.to_latex('grid3.tex')